In [0]:
import json
from includes.UpdateDashboardParams import update_parameters
import os

In [0]:
# Use relative date format for Databricks dashboard date parameters
default_date_min = 'now-90d/d'
default_date_max = 'now/d'

dbutils.widgets.text('catalog_name', defaultValue='')
dbutils.widgets.text('schema_name', defaultValue='')
dbutils.widgets.text('dashboard_id', defaultValue='')
dbutils.widgets.text('date_range_min', defaultValue=default_date_min)
dbutils.widgets.text('date_range_max', defaultValue=default_date_max)
dbutils.widgets.text('space_id_default', defaultValue='ALL')
dbutils.widgets.text('skip_parameter_update', defaultValue='False')
dbutils.widgets.text('deployed', defaultValue='false')

catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
dashboard_id = dbutils.widgets.get('dashboard_id')
date_range_min = dbutils.widgets.get('date_range_min') or default_date_min
date_range_max = dbutils.widgets.get('date_range_max') or default_date_max
space_id_default = dbutils.widgets.get('space_id_default') or 'ALL'
skip_update = dbutils.widgets.get('skip_parameter_update').lower() == 'true'

workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
deployed = dbutils.widgets.get('deployed').lower() == 'true'

assert catalog_name and schema_name, 'Catalog name and schema name inputs must be provided'

print(f"Parameters:")
print(f"  catalog_name: {catalog_name}")
print(f"  schema_name: {schema_name}")
print(f"  dashboard_id: {dashboard_id}")
print(f"  date_range_min: {date_range_min}")
print(f"  date_range_max: {date_range_max}")
print(f"  space_id_default: {space_id_default}")

if skip_update:
  dbutils.notebook.exit('Skipping dashboard parameter update')

In [0]:
# Note if you are developing you want to use ./dashboards/. The ../../resources are the dashboards created during bundle deployment
if deployed:
  dashboard_root_path = '../../resources/'
else:
  dashboard_root_path = './dashboards/'
all_dashboards = [f for f in os.listdir(dashboard_root_path) if f.endswith('.lvdash.json')]

for dashboard in all_dashboards:
  target_file = os.path.join(dashboard_root_path, dashboard)
  print(f"Attempting to update: {target_file}")
  with open(target_file, 'r') as f:
    dashboard_json = json.load(f)
  
  # Update parameters with all new values
  updated_json = update_parameters(
    dashboard_json, 
    catalog_name, 
    schema_name, 
    workspace_url, 
    dashboard_id,
    date_range_min,
    date_range_max,
    space_id_default
  )
  
  with open(target_file, 'w') as f:
    json.dump(updated_json, f, indent=2)
  
  print(f'Updated dashboard: {target_file}')
  print(f'  - catalog: {catalog_name}')
  print(f'  - schema: {schema_name}')
  print(f'  - workspace_url: {workspace_url}')
  print(f'  - dashboard_id: {dashboard_id}')
  print(f'  - date_range: {date_range_min} to {date_range_max}')
  print(f'  - space_id_default: {space_id_default}')
  print()